In [4]:
# --- Funnel Builder ---
import pandas as pd
import numpy as np
# Create binary flags per session
web_events = pd.read_csv('web_events.csv')
funnel_flags = web_events.pivot_table(
    index='session_id',
    columns='event_type',
    aggfunc='size',
    fill_value=0
).reset_index()

# Convert count to binary presence flags
funnel_flags = funnel_flags.assign(
    product_view=lambda df: (df.get('product_view', 0) > 0).astype(int),
    add_to_cart=lambda df: (df.get('add_to_cart', 0) > 0).astype(int),
    checkout_started=lambda df: (df.get('payment_attempted', 0) > 0).astype(int),
    order_completed=lambda df: (df.get('order_completed', 0) > 0).astype(int)
)[['session_id', 'product_view', 'add_to_cart', 'checkout_started', 'order_completed']]

# Conversion rates
total_sessions = len(funnel_flags)

conversion = {
    'view_to_cart': funnel_flags['add_to_cart'].sum() / funnel_flags['product_view'].sum() * 100,
    'cart_to_checkout': funnel_flags['checkout_started'].sum() / funnel_flags['add_to_cart'].sum() * 100,
    'checkout_to_order': funnel_flags['order_completed'].sum() / funnel_flags['checkout_started'].sum() * 100,
}

print("Stage-to-Stage Conversion Rates (%):")
for stage, rate in conversion.items():
    print(f"{stage}: {rate:.2f}%")

Stage-to-Stage Conversion Rates (%):
view_to_cart: 67.06%
cart_to_checkout: 84.27%
checkout_to_order: 68.67%
